# Validação Pydantic - example.py

In [73]:
from enum import auto, IntFlag
from typing import Any

# importações
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)
# definie o papéis do usuário 
class Role(IntFlag):
    Author = auto() 
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer # O administrador vai para esses agentes


In [74]:
class User(BaseModel): # A classe user herda o sistema de validação do Pydantic
    # Esses são os campos que fazem parte da classe de validação
    name: str = Field(examples=["Arjan"]) 
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True, # não pode mudar depois da validação
    )
    password: SecretStr = Field( # SecretStr senha será printada de forma escondida 
        examples=["Password123"], description="The password of the user" # descrição: interessante para documentação
    )
    role: Role = Field(default=None, description="The role of the user")


In [75]:
# tentando criar o user
def validate(data: dict[str, Any]) -> None: 
    try:
        user = User.model_validate(data) # chama o modelo de validação
        print(user)
    except ValidationError as e:
        print("User is invalid")
        for error in e.errors():
            print(error)

In [76]:
# demonstração com dados corretos e incorretos para validação
def main() -> None:
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
    bad_data = {"email": "<bad data>", "password": "<bad data>"} # dados incorretos

    validate(good_data)
    validate(bad_data)

In [77]:
if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.11/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


# Validação Personalizada Pydantic

In [78]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)
# REGEX para validação de nomes e senhas
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8



In [79]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    # Campos de validação
    @field_validator("name") # chama a função que valida o campo, passando o campo que deve ser validado
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v): # verificando se o REGEX corresponde ao que foi obtido
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v # retorna o valor

    @field_validator("role", mode="before") # se definir como depois, você pode escolher o tipo de entrada que deseja  - pois ainda não foi validado
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        # validações caso falte algum campo
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest() # faz o hash
        return v

In [80]:
# validação dos dados
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data) # cria um usuário a partir do modelo de dados
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

In [81]:
# simula a validação de usuários
def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )
    
    # looping passando por cada cenário
    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()
        

In [82]:
if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

# Serialização de dados com Pydantic

In [83]:
import enum
import hashlib
import re
from typing import Any, Self

In [84]:
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)
# definindo o regex
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [85]:
# definindo o conjunto de permissões combinando o pedido de multiplas permissões
class Role(enum.IntFlag):
    User = 0 # permissao base, ou nenhuma
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8 # permissao total

class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True # senha não será incluida ao serializar os dados do usuário
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )

# validação dos campos
    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )
 
    @model_validator(mode="before") # serialização antes
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after") # serialização pís
    def validate_user_post(self, v: Any) -> Self:
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json") # serializador de campo
    @classmethod
    def serialize_role(cls, v) -> str:
        return v.name # nome da função deve ser retornado para que possa ser armazenado em JSON

    # definindo como os modelos estão sendo serializados
    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)


In [86]:
def main() -> None:
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"), # coisas que podem ser excluidas da serialização
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")


if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


# Validação de dados Pydantic com FastAPI

In [87]:
from datetime import datetime
from typing import Optional
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

app = FastAPI()

In [88]:
class User(BaseModel): # modelo de usuário do Pydantic
    model_config = { # configuração específica para APIs com Pydantic
        "extra": "forbid", # proibe adição de atributos extras, um terceiro não pode adicionar atributos a API
    }
    __users__ = []
    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        return str(id)


In [89]:
# Rotas 
@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    return list(User.__users__)


@app.post("/users", response_model=User)
async def create_user(user: User):
    User.__users__.append(user)
    return user


@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    try:
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})

In [ ]:
# testes automatizados usando o TestClient do FastAPI.
def main() -> None: # Executando testes usando fastAPI
    with TestClient(app) as client:
        for i in range(5):
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )
            assert response.status_code == 200
            assert response.json()["name"] == f"User {i}", (
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id"

            user = User.model_validate(response.json())
            assert str(user.id) == response.json()["id"], "The id should be the same"
            assert user.signup_ts, "The signup timestamp should be set"
            assert user.friends == [], "The friends list should be empty"
            assert user.blocked == [], "The blocked list should be empty"

        response = client.get("/users")
        assert response.status_code == 200, "Response code should be 200"
        assert len(response.json()) == 5, "There should be 5 users"

        response = client.post(
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()["id"], "The id should be the same"
        assert user.signup_ts, "The signup timestamp should be set"
        assert user.friends == [], "The friends list should be empty"
        assert user.blocked == [], "The blocked list should be empty"

        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}")
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"name": "User 6", "email": "wrong"})
        assert response.status_code == 422, "The email address is should be invalid"


if __name__ == "__main__":
    main()